# Deep Learning


**TODO**

1. ~Ho to define models~ 

    * ~Helow word simple regression~
    * ~define models with sequence~
    
1. Callbacks 
    * How to stop when reach specific acc or perfromance 
    * How to save best model during 
    * How to save models archicterue, parameters, optmizer parameters (everything) (see my code in dialects)
    * How to load saved models 

1. ~Vectorization operations and loss functions~

    * ~create ones and zeros and get shape~
    * ~matrix n vector mutliplications~
    * ~reshape matrix~
    * ~How to encode one-hot~
    
1. Code for main loss functions and when to use

    * binnary cross entropy
    * categorical cross entropy
    * sparse categorical cross entropy
    * mae
    * mse 

1. Learning curves 
    * Code to plot learning curves in notebook using matplotlib
    * Code to use tensorboard. Add how to use and interpret the histogram of weight intensorboard

1. Code to use Genrators
    * Split folders in train and val so Generators can read 
    * ImageDataGenerator with data augmentation
    * Use ImageDataGenerator to save aigmented data

refs:
* https://blog.goodaudience.com/artificial-neural-networks-explained-436fcf36e75


## Tensorflow 


In [8]:
import math
import numpy as np
import h5py
import matplotlib.pyplot as plt
import tensorflow as tf
#from tensorflow.python.framework import ops
#from tf_utils import load_dataset, random_mini_batches, convert_to_one_hot, predict

#slim = tf.contrib.slim

%matplotlib inline
np.random.seed(1)

import IPython
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

print(tf.__version__)

2.0.0


## Hello word (regression)

In [72]:
# the data
# rule that We need to find is y = 2x - 1
xs = np.array([-1.0, 0.0, 1.0, 2.0, 3.0, 4.0], dtype=float)
ys = np.array([-3.0, -1.0,1.0,3.0,5.0,7.0], dtype=float)

# Define a model with one neuron
model = tf.keras.Sequential([tf.keras.layers.Dense(units=1, input_shape=[1])])

# define loss func and optimizer 
model.compile(optimizer='sgd', loss='mean_squared_error')

# train or learn the rule
model.fit(xs, ys, epochs=500, verbose=0)

print("Checking")
x = 3.0
print(f"x: {x}; expected: {2.0*x-1}; prediction: {model.predict([x])}")

print("unseen data")
x = 10.00
print(f"x: {x}; expected: {2.0*x-1}; prediction: {model.predict([x])}")

Checking
x: 3.0; expected: 5.0; prediction: [[5.000253]]
unseen data
x: 10.0; expected: 19.0; prediction: [[18.982594]]


## Tensor basic operations 

https://www.tensorflow.org/guide/tensor


In [52]:
import numpy as np

print(f"vector add: {tf.add([1, 2], [3, 4])}")
print(f"sum all elemenst: {tf.reduce_sum([1, 2, 3])}")

mymat = tf.Variable([[7],[11]], tf.int16)
print(f"mymat: {mymat.value()}")

# batch x height x width x color
batch_imgs = tf.zeros([10, 299, 299, 3])  
print(f"batch_imgs: {batch_imgs.shape}")

print()

y_hat = tf.constant(36)            # Define y_hat constant. Set to 36.
y = tf.constant(39)   

loss = tf.Variable((y - y_hat)**2)

print(f"loss: {loss.value()}")

print()
print("numpy compatibility")

ndarray = np.ones([3, 3])

print("Tensor convert to numpy")
tensor = tf.multiply(ndarray, 10)
print(f"tensor: {tensor}")

print()
print("And Tensors converts to numpy")
print(f" add scalr to tensor and get a numpy array:  {np.add(tensor, 1)}")

print()
print("The .numpy() method explicitly converts a Tensor to a numpy array")
print(tensor.numpy())

print()
print(f"tensor ones: {tf.Variable(np.ones([3, 3]))}")

print()
print("Reshaping")

x = tf.reshape(np.array([1,2,3,4,5,6,7,8,9]), [3, 3])
print(f"x shape: {x.shape}")
print(f"x: {x.numpy()}")

# -1 means infere this dimension
y = tf.reshape(np.array([1,2,3,4,5,6,7,8,9,10,11,12]), [2, -1])
print(f"y shape: {y.shape}")
print(f"x: {y.numpy()}")

print()
print("Transpose")
# Transpose
print(f"y shape: {y.shape}")
print(f"y.T: {tf.transpose(y).shape}")


vector add: [4 6]
sum all elemenst: 6
mymat: [[ 7]
 [11]]
batch_imgs: (10, 299, 299, 3)

loss: 9

numpy compatibility
Tensor convert to numpy
tensor: [[10. 10. 10.]
 [10. 10. 10.]
 [10. 10. 10.]]

And Tensors converts to numpy
 add scalr to tensor and get a numpy array:  [[11. 11. 11.]
 [11. 11. 11.]
 [11. 11. 11.]]

The .numpy() method explicitly converts a Tensor to a numpy array
[[10. 10. 10.]
 [10. 10. 10.]
 [10. 10. 10.]]

tensor ones: <tf.Variable 'Variable:0' shape=(3, 3) dtype=float64, numpy=
array([[1., 1., 1.],
       [1., 1., 1.],
       [1., 1., 1.]])>

Reshaping
x shape: (3, 3)
x: [[1 2 3]
 [4 5 6]
 [7 8 9]]
y shape: (2, 6)
x: [[ 1  2  3  4  5  6]
 [ 7  8  9 10 11 12]]

Transpose
y shape: (2, 6)
y.T: (6, 2)


## Layers: Understanding, Debbuging  and analyze the output 

In [11]:
X = np.array([range(10),range(10)])
X[1,:] = X[1,:] + 1 
X = np.reshape(X,(2,10,1))
X.shape
X.T

(2, 10, 1)

array([[[ 0,  1],
        [ 1,  2],
        [ 2,  3],
        [ 3,  4],
        [ 4,  5],
        [ 5,  6],
        [ 6,  7],
        [ 7,  8],
        [ 8,  9],
        [ 9, 10]]])

* GlobalAveragePooling1D and MaxPool1D

    * https://www.tensorflow.org/api_docs/python/tf/keras/layers/GlobalAveragePooling1D
    * https://www.tensorflow.org/api_docs/python/tf/keras/layers/MaxPool1D
    * https://www.tensorflow.org/api_docs/python/tf/keras/layers/Conv1D

```python
tf.keras.layers.MaxPool1D(
    pool_size=2, strides=None, padding='valid', data_format='channels_last',
    **kwargs
)


# filters is the number of neurons
tf.keras.layers.Conv1D(
    filters, kernel_size, strides=1, padding='valid', data_format='channels_last',
    dilation_rate=1, activation=None, use_bias=True,
    kernel_initializer='glorot_uniform', bias_initializer='zeros',
    kernel_regularizer=None, bias_regularizer=None, activity_regularizer=None,
    kernel_constraint=None, bias_constraint=None, **kwargs
)

```

In [23]:
print("Conv1d")

_kernel_init_coefs = np.array([-1, 0, 1])
_kernel_init_coefs = tf.initializers.constant(_kernel_init_coefs)


model3 = tf.keras.Sequential([
        tf.keras.layers.Conv1D(1, kernel_size=3, kernel_initializer= _kernel_init_coefs, input_shape=(10,1) )
    ])

model3.summary()
y3 = model3.predict(X)
y3.shape

X.T

# Expected all 2 values becasue: 
# 0*-1 + 1*0 + 2*1 = 2
# 1*-1 + 2*0 + 3*1 = 2
# ... 
y3.T

Conv1d
Model: "sequential_14"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv1d_7 (Conv1D)            (None, 8, 1)              4         
Total params: 4
Trainable params: 4
Non-trainable params: 0
_________________________________________________________________


(2, 8, 1)

array([[[ 0,  1],
        [ 1,  2],
        [ 2,  3],
        [ 3,  4],
        [ 4,  5],
        [ 5,  6],
        [ 6,  7],
        [ 7,  8],
        [ 8,  9],
        [ 9, 10]]])

array([[[2., 2.],
        [2., 2.],
        [2., 2.],
        [2., 2.],
        [2., 2.],
        [2., 2.],
        [2., 2.],
        [2., 2.]]], dtype=float32)

In [24]:
print("GlobalAveragePooling1D")
model1 = tf.keras.Sequential([
        tf.keras.layers.GlobalAveragePooling1D(input_shape=(10,1) )
    ])
model1.summary()
y1 = model1.predict(X)
y1.shape
y1.T

print()
print("MaxPool1D")

model2 = tf.keras.Sequential([
        tf.keras.layers.MaxPool1D(pool_size=2, input_shape=(10,1) )
    ])

model2.summary()
y2 = model2.predict(X)
y2.shape

y2.T

GlobalAveragePooling1D
Model: "sequential_15"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
global_average_pooling1d_5 ( (None, 1)                 0         
Total params: 0
Trainable params: 0
Non-trainable params: 0
_________________________________________________________________


(2, 1)

array([[4.5, 5.5]], dtype=float32)


MaxPool1D
Model: "sequential_16"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
max_pooling1d_2 (MaxPooling1 (None, 5, 1)              0         
Total params: 0
Trainable params: 0
Non-trainable params: 0
_________________________________________________________________


(2, 5, 1)

array([[[ 1.,  2.],
        [ 3.,  4.],
        [ 5.,  6.],
        [ 7.,  8.],
        [ 9., 10.]]], dtype=float32)

In [28]:
from tensorflow.keras import models

_kernel_init_coefs = np.array([1, 1, 1])
_kernel_init_coefs = tf.initializers.constant(_kernel_init_coefs)

model = tf.keras.Sequential([
        tf.keras.layers.Conv1D(1, kernel_size=3, kernel_initializer= _kernel_init_coefs, input_shape=(10,1) ),
        tf.keras.layers.MaxPool1D(pool_size=2),
        tf.keras.layers.GlobalAveragePooling1D()
    ])

model.summary()

layer_outputs = [layer.output for layer in model.layers]
activation_model = tf.keras.models.Model(inputs = model.input, outputs = layer_outputs)

for l in range(3):
    
    print(f"Layer: {l}")
    
    f1 = activation_model.predict(X)[l]
    print(f"output shape: {f1.shape}")
    print(f1.T)

Model: "sequential_20"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv1d_11 (Conv1D)           (None, 8, 1)              4         
_________________________________________________________________
max_pooling1d_6 (MaxPooling1 (None, 4, 1)              0         
_________________________________________________________________
global_average_pooling1d_9 ( (None, 1)                 0         
Total params: 4
Trainable params: 4
Non-trainable params: 0
_________________________________________________________________
Layer: 0
output shape: (2, 8, 1)
[[[ 3.  6.]
  [ 6.  9.]
  [ 9. 12.]
  [12. 15.]
  [15. 18.]
  [18. 21.]
  [21. 24.]
  [24. 27.]]]
Layer: 1
output shape: (2, 4, 1)
[[[ 6.  9.]
  [12. 15.]
  [18. 21.]
  [24. 27.]]]
Layer: 2
output shape: (2, 1)
[[15. 18.]]


### Encoding

* One hot encoding

https://www.tensorflow.org/api_docs/python/tf/one_hot

<img src="images/onehot.png" width="800" align="left"/>

In [58]:
from tensorflow.keras.utils import to_categorical

test_labels = tf.constant([1,2, 3, 0, 2, 1])

y = to_categorical(test_labels)
y.shape

k = 1
print(f"labels: {test_labels[k]}; one-hot encode: {y[k,:]}")
y

(6, 4)

labels: 2; one-hot encode: [0. 0. 1. 0.]


array([[0., 1., 0., 0.],
       [0., 0., 1., 0.],
       [0., 0., 0., 1.],
       [1., 0., 0., 0.],
       [0., 0., 1., 0.],
       [0., 1., 0., 0.]], dtype=float32)

In [49]:
indices = [0, 1, 2]
depth = 3

# ?tf.one_hot
tf.one_hot(indices, depth)  # output: [3 x 3]

<tf.Tensor: id=825, shape=(3, 3), dtype=float32, numpy=
array([[1., 0., 0.],
       [0., 1., 0.],
       [0., 0., 1.]], dtype=float32)>

### Train 


- **Training set**: 1080 pictures (64 by 64 pixels) of signs representing numbers from 0 to 5 (180 pictures per number).
- **Test set**: 120 pictures (64 by 64 pixels) of signs representing numbers from 0 to 5 (20 pictures per number).


In [ ]:
def load_dataset():
    
    train_dataset = h5py.File('data/train_signs.h5', "r")
    
    train_set_x_orig = np.array(train_dataset["train_set_x"][:]) # your train set features
    train_set_y_orig = np.array(train_dataset["train_set_y"][:]) # your train set labels

    test_dataset = h5py.File('data/test_signs.h5', "r")
    test_set_x_orig = np.array(test_dataset["test_set_x"][:]) # your test set features
    test_set_y_orig = np.array(test_dataset["test_set_y"][:]) # your test set labels

    classes = np.array(test_dataset["list_classes"][:]) # the list of classes
    
    train_set_y_orig = train_set_y_orig.reshape((1, train_set_y_orig.shape[0]))
    test_set_y_orig = test_set_y_orig.reshape((1, test_set_y_orig.shape[0]))
    
    return train_set_x_orig, train_set_y_orig, test_set_x_orig, test_set_y_orig, classes

X_train_orig, Y_train_orig, X_test_orig, Y_test_orig, classes = load_dataset()

In [ ]:
print(np.shape(classes))
print(classes)

# See example of image
index = 0
plt.imshow(X_train_orig[index])
print ("y = " + str(np.squeeze(Y_train_orig[:, index])))

* preprocessing the images

In [ ]:
def convert_to_one_hot(Y, C):
    Y = np.eye(C)[Y.reshape(-1)].T
    return Y

# Flatten the training and test images
X_train_flatten = X_train_orig.reshape(X_train_orig.shape[0], -1).T
X_test_flatten = X_test_orig.reshape(X_test_orig.shape[0], -1).T

# Normalize image vectors
X_train = X_train_flatten / 255.
X_test = X_test_flatten / 255.

# Convert training and test labels to one hot matrices
Y_train = convert_to_one_hot(Y_train_orig, 6)
Y_test = convert_to_one_hot(Y_test_orig, 6)

print("number of training examples = " + str(X_train.shape[1]))
print("number of test examples = " + str(X_test.shape[1]))

print()
print('64*64*3 = 12288')
print("X_train shape: " + str(X_train.shape))
print("Y_train shape: " + str(Y_train.shape))
print("X_test shape: " + str(X_test.shape))
print("Y_test shape: " + str(Y_test.shape))

print()
print ("Y_test (1st 5) = \n" + str(np.squeeze(Y_test[:,0:5])))

## Loss and metrics 

ref: https://www.tensorflow.org/api_docs/python/tf/keras/metrics

### MAE and MSE


In [32]:
y_true = np.array([1  ,0, 1, 0])
y_pred = np.array([0.5,0, 1, 0])

mse = tf.keras.losses.MSE(y_true, y_pred).numpy()
mae = tf.keras.losses.MAE(y_true, y_pred).numpy()

print(f"mse: {mse}")
print(f"mae: {mae}")


mse: 0.0625
mae: 0.125


### Cross entropy

```python

# binary 
tf.keras.losses.binary_crossentropy(
    y_true, y_pred, from_logits=False, label_smoothing=0
)
```

* binary cross entropy

$
- \frac{1}{m}  \sum_{i = 1}^m  \large ( \small y^{(i)} \log (\hat{y}^{(i)}) + (1-y^{(i)})\log (1-\hat{y}^{(i)} \large ) \small\tag{1}
$


In [56]:
y_true = tf.constant([1,0, 1, 0])

# y_pred must be float because it means probabilities
y_pred = tf.constant([0.3,0, 0.9, 0])

_loss = tf.keras.losses.binary_crossentropy(y_true, y_pred)

print(f"loss: {_loss}")

loss: 0.3273332118988037



```python
# categorical crossentropy
tf.keras.losses.categorical_crossentropy(
    y_true, y_pred, from_logits=False, label_smoothing=0
)

# sparse
tf.keras.losses.sparse_categorical_crossentropy(
    y_true, y_pred, from_logits=False, axis=-1
)

# Use the loss function (approx. 1 line)
cost = tf.nn.sigmoid_cross_entropy_with_logits(logits=z, labels=y)

```


In [59]:
## TODO needs to be reviewed 
y_true = to_categorical(tf.constant([1,0, 1, 0]))

# y_pred must be float because it means probabilities
y_pred = to_categorical(tf.constant([0.3,0, 0.9, 0]))

_loss = tf.keras.losses.categorical_crossentropy(y_true, y_pred)

print(f"loss: {_loss}")

loss: [1.1920929e-07 1.1920929e-07 1.1920929e-07 1.1920929e-07]


In [55]:
## TODO needs to be reviewed 
y_true = tf.constant([1,2, 1, 0])



# y_pred must be float because it means probabilities
y_pred = tf.constant([ [0.0,0.9, 0.1],
                       [0.0,0.3, 0.7],
                       [0.5,0.9, 0.5],
                       [0.7,0.1, 0.2]
                     ])

y_pred.shape
y_pred

_loss = tf.keras.losses.categorical_crossentropy(y_true,y_pred)

print(f"loss: {_loss}")

TensorShape([4, 3])

<tf.Tensor: id=1164, shape=(4, 3), dtype=float32, numpy=
array([[0. , 0.9, 0.1],
       [0. , 0.3, 0.7],
       [0.5, 0.9, 0.5],
       [0.7, 0.1, 0.2]], dtype=float32)>

InvalidArgumentError: Incompatible shapes: [4] vs. [4,3] [Op:Mul] name: mul/


* sigmoid_cross_entropy_with_logits

```python

# Use the loss function (approx. 1 line)
cost = tf.nn.sigmoid_cross_entropy_with_logits(logits=z, labels=y)

```

$
- \frac{1}{m}  \sum_{i = 1}^m  \large ( \small y^{(i)} \log \sigma(z^{[2](i)}) + (1-y^{(i)})\log (1-\sigma(z^{[2](i)})\large )\small\tag{2}
$




### Regression 

#### Define model network

#### Train


### Convolutional (TODO)

In [ ]:
from datasets import flowers

### NLP 

### Time Series 